In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import gradio as gr
import base64
from io import BytesIO
import tempfile
import subprocess
from pydub import AudioSegment
from pathlib import Path
from tempfile import NamedTemporaryFile
import time

In [2]:
os.chdir('/Users/kerem/VSProjects/llm_engineering/week2/Booking Agent')

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists


In [4]:
def get_system_prompt():
        with open('system_prompt.txt', 'r', encoding='utf-8') as file:
            system_prompt = file.read()
        return system_prompt
system_message = get_system_prompt()
print(system_message)

-You are a helpful assistant for an Airline called AcarAirlines.  
-Always be accurate. If you don't know the answer, say that there is no avaliable tickets right now and ask another destination.
-Provide the user with all the information you have. 
-If the user claims that the prices are expensive make an %20 percent discount, do not mention this until they say it is expensive.
-Be intimate, make jokes.


In [5]:
DB_PATH = Path("ticket_database.json")

def load_ticket_data():
    with DB_PATH.open('r') as f:
        return json.load(f)
        
def save_ticket_data(data):
    with NamedTemporaryFile('w', delete=False, dir=DB_PATH.parent) as tf:
        json.dump(data, tf, ensure_ascii=False, indent=4)
        tmp_name = tf.name
    os.replace(tmp_name, DB_PATH)         
         

def get_ticket_information(destination_city):
    print(f"Tool get_ticket_information called for {destination_city}")
    city = destination_city.lower()
    data = load_ticket_data()
    info = data.get(city)
    if info:
        return (
            f"Destination: {destination_city.title()}\n"
            f"Price: {info['price']}\n"
            f"Airline: {info['airline']}\n"
            f"Duration: {info['duration']}\n"
            f"Availability {info['availability']}\n"
            
        )
    else:
        return "Unknown destination."
def book_ticket(destination_city):
    print(f"Tool book_ticket called for {destination_city}")
    city = destination_city.lower()
    data = load_ticket_data()
    info = data.get(city)
    if not info:
        return "Unknown destination."
    available = int(info.get("availability", 0))
    if available < 1:
        return "Not enough seats"
    info["availability"] = str(available - 1)
    data[city] = info
    save_ticket_data(data)
    return "Booking was successful"
    
    
print(get_ticket_information("Tokyo"))
print(book_ticket("Tokyo"))
print(get_ticket_information("Tokyo"))

Tool get_ticket_information called for Tokyo
Destination: Tokyo
Price: $1400
Airline: Japan Airlines
Duration: 14h 20m
Availability 37

Tool book_ticket called for Tokyo
Booking was successful
Tool get_ticket_information called for Tokyo
Destination: Tokyo
Price: $1400
Airline: Japan Airlines
Duration: 14h 20m
Availability 36



In [6]:
info_function = {
    "name": "get_ticket_information",
    "description":(
        "Get the information of a ticket to the destination city.\n"
        "Call this whenever you need to know the details of the ticket, for example when a customer asks:\n"
        "'How much is a ticket to this city?', 'How long is a flight to this city?'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
booking_function = {
    "name": "book_ticket",
    "description":(
        "Book the ticket to the destination city.\n"
        "Call this whenever you need to book the ticket, for example when a customer says:\n"
        "'Let's book it.', 'I want to buy this ticket.'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": info_function},{"type": "function", "function": booking_function}]

In [7]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response= handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [8]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    name = json.dumps(tool_call.function.name)
    if name.replace('"','') == "get_ticket_information":
        info = get_ticket_information(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"info": info}),
            "tool_call_id": tool_call.id
        }
        return response
    elif name.replace('"','') == "book_ticket": 
        info = book_ticket(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"info": info}),
            "tool_call_id": tool_call.id
        }
        return response

In [9]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
